In [36]:
import requests as req
import duckdb
import pandas as pd
from datetime import datetime, timedelta
import time


In [56]:
def get_jeopardy_game(month, day, year, max_retries=3, base_delay=1):
    game_data = []
    failed_dates = []
    month_str = str(month).zfill(2)
    day_str = str(day).zfill(2)

    for round_num in range(1, 3):
        url = f"https://jarchive-json.glitch.me/glitch/{month_str}/{day_str}/{year}/{round_num}"
        
        for attempt in range(max_retries):
            try:
                response = req.get(url, timeout=10)
                response.raise_for_status()
                
                # Check for "no game" message in response
                data = response.json()
                if isinstance(data, dict) and "message" in data:
                    if "does not have a game" in data["message"]:
                        return None  # Indicate no game available
                
                game_data.append(data)
                break
                
            except req.exceptions.RequestException as e:
                delay = base_delay * (2 ** attempt)
                if attempt < max_retries - 1:
                    print(f"Attempt {attempt + 1} failed for round {round_num}. Retrying in {delay} seconds...")
                    time.sleep(delay)
                else:
                    failed_dates.append(f"{year}-{month_str}-{day_str}")
        
    if not game_data:
        raise Exception("Failed to fetch any game data")
        
    return game_data, failed_dates

In [47]:
def store_jeopardy_game(month, day, year):
    # Fetch the game data
    game_data = get_jeopardy_game(month, day, year)
    
    # Transform data into a flat structure
    flattened_data = []
    game_date = datetime(year, month, day).date()  # Convert to date instead of datetime
    
    for round_num, round_data in enumerate(game_data, 1):
        for category, clues in round_data.items():
            for clue in clues:
                flattened_data.append({
                    'game_date': game_date,
                    'round': round_num,
                    'category': category,
                    'clue': clue['clue'],
                    'answer': clue['answer']
                })
    
    # Convert to DataFrame
    df = pd.DataFrame(flattened_data)
    
    # Initialize DuckDB and create table if it doesn't exist
    conn = duckdb.connect('jeopardy.db')
    
    # Create table if it doesn't exist
    conn.execute("""
        CREATE TABLE IF NOT EXISTS jeopardy_clues (
            game_date DATE,
            round INTEGER,
            category VARCHAR,
            clue VARCHAR,
            answer VARCHAR,
            PRIMARY KEY (game_date, round, category, clue)
        )
    """)
    
    # Convert DataFrame to DuckDB table and insert
    conn.execute("INSERT OR IGNORE INTO jeopardy_clues SELECT * FROM df")
    conn.commit()  # Add explicit commit
    conn.close()
    
    return len(flattened_data)

In [51]:
def store_jeopardy_games_range(start_date, end_date):
    current_date = start_date
    total_clues = 0
    
    while current_date <= end_date:
        try:
            clues_stored = store_jeopardy_game(
                current_date.month,
                current_date.day,
                current_date.year
            )
            print(f"Stored {clues_stored} clues for {current_date.date()}")
            total_clues += clues_stored
            
        except Exception as e:
            print(f"Failed to fetch/store game for {current_date.date()}: {str(e)}")
            
        current_date += timedelta(days=1)
    
    return total_clues


Stored 60 clues for 2023-10-23
Stored 60 clues for 2023-10-24
Stored 60 clues for 2023-10-25
Stored total of 180 clues


In [ ]:
# Example usage:
start = datetime(2023, 10, 23)
end = datetime(2023, 10, 25)
total = store_jeopardy_games_range(start, end)
print(f"Stored total of {total} clues")

In [55]:
# Query example
conn = duckdb.connect('jeopardy.db')
result = conn.execute("""
    SELECT * 
    FROM jeopardy_clues 
    WHERE game_date = DATE '2023-10-25'
    LIMIT 5
""").fetchall()
conn.close()
result

[(datetime.date(2023, 10, 25),
  1,
  'PRODUCE',
  'Big on vitamins A & C, this leafy vegetable also has a lot of fiber & acts as a mild laxative; look out, sailor man!',
  'spinach'),
 (datetime.date(2023, 10, 25),
  1,
  'PRODUCE',
  'In 2008 a pair of Brits were out of their gourd, growing a 1,457-lb. one of these; smashing!',
  'a pumpkin'),
 (datetime.date(2023, 10, 25),
  1,
  'PRODUCE',
  'Dr. Thomas Welch is credited with making the first unfermented juice from this fruit',
  'a grape'),
 (datetime.date(2023, 10, 25),
  1,
  'PRODUCE',
  'When life gives you the Meyer type of this tree, expect plenty of fruit year-round',
  'a lemon'),
 (datetime.date(2023, 10, 25),
  1,
  'PRODUCE',
  'The name of this cylindrical vegetable means "little squashes" in Italian',
  'zucchini')]